In [3]:
import gensim
import pandas as pd
import jieba
import re
import stanfordcorenlp
import math
import numpy as np
import random

## Data preprocessing

In [4]:
data_source = "./sqlResult_1558435.csv"

In [5]:
data = pd.read_csv(data_source,encoding='gb18030')

In [6]:
data = data.fillna('')  #缺失数据填充

In [7]:
def cut(string): return ' '.join(jieba.cut(string))

In [8]:
cut("你今天运动了吗")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wy\AppData\Local\Temp\jieba.cache
Loading model cost 0.880 seconds.
Prefix dict has been built successfully.


'你 今天 运动 了 吗'

In [9]:
content = data['content'].tolist()

In [10]:
content[0]

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n'

In [8]:
def token(string):
    return re.findall(r'[\d|\w]+',string)

In [9]:
token("这是一个测试\n\n\n")

['这是一个测试']

In [10]:
news_content = [token(n) for n in content]  

In [11]:
news_content = [''.join(n) for n in news_content]

In [12]:
news_content = [cut(n) for n in news_content]

In [13]:
news_content[0]

'此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息'

## TF-IDF

Term frequency: $tf_{t,d} = log_{10} count(t,d)+1$  
Inverse document frequency: $idf_{t} = log_{10} (N/df_{t})$  
TF-idfweighted value: $w_{t,d} = tf_{t,d} x idf_{t}$   

N - 文档总数  
df_t - 出现t这个词的文档数目

In [14]:
def document_frequency(word):  #计算出现word的文档次数
    return sum(1 for n in news_content if word in n)

In [15]:
document_frequency('小米')

100

In [16]:
def idf(word):
    return math.log10(len(news_content)/document_frequency(word))

In [17]:
idf("的")

0.1051466115514474

In [18]:
idf("小米")

2.9523613237924735

In [19]:
def tf(word,document): #计算文档中word出现的次数，没有用log
    words = document.split()
    return sum(1 for w in words if w==word)

In [20]:
tf("的",news_content[0])

2

In [21]:
def tf_idf(word,document):  
    return tf(word,document)*idf(word)

In [22]:
tf_idf("的",news_content[0])

0.2102932231028948

In [23]:
def get_keywords_of_a_document(document):
    words = set(document.split())
    tfidf = [
        (w,tf_idf(w,document)) for w in words
    ]
    tfidf = sorted(tfidf,key=lambda x: x[1],reverse=True)
    return tfidf

In [24]:
get_keywords_of_a_document(news_content[1])[:5]

[('骁龙', 14.28860032832347),
 ('Windows10', 12.14781401040159),
 ('桌面', 8.674990450316585),
 ('高通', 6.392972936239964),
 ('835', 6.363018624300659)]

In [25]:
news_content[1]

'骁龙 835 作为 唯一 通过 Windows10 桌面 平台 认证 的 ARM 处理器 高通 强调 不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心 相反 他们 正 联手 微软 找到 一种 适合 桌面 平台 的 兼顾 性能 和 功耗 的 完美 方案 报道 称 微软 已经 拿到 了 一些 新 的 源码 以便 Windows10 更好 地 理解 biglittle 架构 资料 显示 骁龙 835 作为 一款 集成 了 CPUGPU 基带 蓝牙 WiFi 的 SoC 比 传统 的 Wintel 方案 可以 节省 至少 30 的 PCB 空间 按计划 今年 Q4 华硕 惠普 联想 将 首发 骁龙 835Win10 电脑 预计 均 是 二合一 形态 的 产品 当然 高通 骁龙 只是 个 开始 未来 也许 还 能 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 等 进入 Windows10 桌面 平台'

### TFIDF Vectorizer

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
vectorizer = TfidfVectorizer(max_features=10000)

In [28]:
sample_num = 50000
sub_samples = news_content[:sample_num]  #取前50000个新闻

In [29]:
X = vectorizer.fit_transform(sub_samples)  #X为50000x10000的稀疏矩阵,10000表示word的个数

In [30]:
X[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [31]:
np.where(X[1].toarray())  # 返回稀疏矩阵中不为0的索引

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([ 100,  281,  360,  372,  472,  578, 1008, 1162, 1244, 1269, 1285,
        1380, 1462, 1886, 2435, 2754, 2760, 2989, 3067, 3353, 3800, 3990,
        4173, 4272, 4429, 4497, 4521, 4529, 4595, 4658, 4961, 5037, 5104,
        5136, 5550, 5717, 5771, 5899, 6035, 6081, 6710, 6971, 7072, 7217,
        7506, 7986, 8025, 8026, 8137, 8207, 8479, 8558, 8799, 9127, 9196,
        9252, 9637, 9744, 9824], dtype=int64))

In [32]:
X[1].toarray()[0][12]

0.0

In [33]:
document_1,document_2 = random.randint(0,1000),random.randint(0,1000)

In [34]:
document_1

529

In [35]:
document_2

152

In [36]:
news_content[document_1]

'英国 每日 电讯报 报道 负责 足球 规则 制定 的 国际足球 协会 理事会 IFAB 正在 考虑 对 现有 足球比赛 的 规则 进行 修改 足球比赛 的 时间 可能 由 现在 的 90 分钟 缩短 为 60 分钟 足球比赛 的 规则 可能 在 不久 的 将来 将 迎来 剧变 IFAB 在 一份 题为 公平 比赛 框架 文件 中 提出 了 12 项 改革 提议 这些 改革 主要 是 出于 3 个 目的 的 考量 其一 是 改善 球员 的 场上 行为 加强 尊重 意识 其二 是 增加 场上 实际 比赛 的 时间 其三 则 是 加强 公平 比赛 和 积极 比赛 的 意识 值得一提的是 60 分钟 的 比赛 时间 将会 是 实际 比赛 时间 不 包括 死球 状态 的 时间 当 比赛 中 出现 死球 状态 时 记录 比赛 用时 的 钟表 将 暂停 现场 负责 计时 的 钟表 将 与 主裁判 佩戴 的 手表 完全 同步 IFAB 在 这份 文件 中 表示 很多 人 都 非常 失望 的 是 本来 90 分钟 的 比赛 时间 但 实际 比赛 时间 少于 60 分钟 我们 这项 提议 就是 为了 抑制 浪费时间 的 行为 主要 是 加速 比赛 进行 的 进程 IFAB 强调 这 12 项 规则 的 改变 有 的 将 直接 施行 不 需要 进行 规则 修改 有 的 需要 进行 试验 有 的 则 需要 进行 讨论 在 需要 进行 讨论 的 提议 类别 中 包括 任意球 角球 以及 球门 球 的 主罚 过程 中 主罚 者 可以 自己 带球 前行 另外 一些 需要 进行 讨论 的 提议 还 包括 主裁判 只会 在 死球 状态 才 会 吹响 中场 和 比赛 结束 的 哨声 而 在 常规 时间 内 主罚 点球 球员 主罚 点球 一旦 被 扑 出后 不 允许 再 进行 补射 而 无需 修改 规则 的 提议 主要 是 抑制 拖延时间 的 行为 主裁判 需要 严格 按照 规则 行事 只 允许 门将 持球 达到 6 秒 在 判罚 点球 和 点球 主罚 的 过程 中 主裁判 必须 停表'

In [37]:
vector_of_document_1 = X[document_1].toarray()[0]

In [38]:
news_content[document_2]

'东方 卫视 明星 旅游 体验 综艺节目 旅途 的 花样 上 周六 播出 了 第三期 在 经历 了 上次 的 撒哈拉 风暴 之后 花样 团 又 接到 了 新 的 任务 参加 摩洛哥 当地 一年一度 盛大 的 玫瑰 节 庆典 当天 一席 红裙 的 张歆艺 和 花样 团 配合默契 凭借 华丽 的 歌舞 表演 惊艳 摩洛哥 不仅如此 在 本期 节目 中 张歆艺 还 难 掩 吃货 本色 为 花样 团煮 起 了 四川火锅 网友 笑称 二姐 的 火锅 底料 已 上线 螺蛳 粉 还 会 远 么 花样 团 惊艳 摩洛哥 张歆艺 排舞 导演 范儿 十足 在 玫瑰 节 庆典 当天 花样 团在 经过 精心 准备 后 直接 变 身 花样 歌舞团 盛装 出道 张歆艺 身穿 一席 热情 似火 的 红裙 亮相 在 表演 前 兴奋 地 表示 终于 登上 了 国际舞台 在 当地 颇具 盛名 的 玫瑰 谷 花样 团用 十分 华丽 但 也 很 考验 默契 度 的 千手观音 开场 在 表演 过程 中 张歆艺 还 不时 为 队友 提示 动作 协调 每个 节奏 点 导演 范儿 十足 开场 舞 结束 后 花样 团选 了 一首 十分 应景 的 热情 的 沙漠 来 展现 热情 惊艳 的 表演 让 他们 在 现场 圈 了 一大 票 摩洛哥 粉丝 张歆 艺为 队友 煮 麻辣火锅 沈腾 招架不住 被 辣晕 在 本期 节目 中 不仅 有 花样 团 的 热辣 表演 也 有 大快朵颐 的 火锅 大餐 作为 团队 中 的 吃货 担当 张歆艺 终于 放出 大招 拿出 了 在 第一期 节目 中 亮相 过 火锅 底料 在 旅途 中为 大家 煮 起 了 四川 麻辣火锅 东北 大哥 沈腾 吃 完 之后 不得不 感叹 死辣 死 辣 的 有 网友 笑称 二姐 的 火锅 底料 已 上线 螺蛳 粉 还 会 远 么'

In [39]:
vector_of_document_2 = X[document_2].toarray()[0]

In [40]:
from scipy.spatial.distance import cosine

In [41]:
def distance(v1,v2): return cosine(v1,v2)

In [42]:
distance(vector_of_document_1,vector_of_document_2)

0.9896810859563955

## TextRank   homework

## LDA

In [43]:
from gensim import corpora, models
import jieba.posseg as jp,jieba  #jp用于词性标注

In [44]:
news = news_content[:100]

In [45]:
flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')

In [46]:
data = pd.read_csv(data_source,encoding='gb18030')

In [47]:
news = data["content"][:100]

In [48]:
#获取停用词表
stop_words = []
with open('./baidu_stopwords.txt',encoding='utf-8') as f:
    for word in f.readlines():
        stop_words.append(word.strip())

In [49]:
words_ls = []
for text in news:
    words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stop_words]
    words_ls.append(words)

In [50]:
dictionary = corpora.Dictionary(words_ls)

In [51]:
#dictionary.doc2bow:词袋模型
corpus = [dictionary.doc2bow(words) for words in words_ls]

https://radimrehurek.com/gensim/models/ldamodel.html

`gensim.models.ldamodel.LdaModel`(corpus=None, num_topics=100, id2word=None, distributed=False, chunksize=2000, passes=1, update_every=1, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, minimum_probability=0.01, random_state=None, ns_conf=None, minimum_phi_value=0.01, per_word_topics=False, callbacks=None, dtype=<class 'numpy.float32'>)  

Parameters:
- `corpus`: 文档向量或稀疏矩阵(num_terms,num_documents)
- `num_topics`: 主题数  
- `id2word`: 词id-词的映射
- `distributed`: 是否计算分布用于加快训练 

Method：
- `get_document_topics(bow, minimum_probability=None, minimum_phi_value=None, per_word_topics=False)`  
得到给定文档的主题分布  

- `get_term_topics(word_id, minimum_probability=None)`  
获取给定词最相关的主题

-`get_topic_terms(topicid, topn=10)`  
获取某个主题下的前topen个词语

In [52]:
#num_topics即为主题数目
lda = models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary,num_topics=5) 

In [53]:
lda.get_document_topics(corpus)

In [54]:
lda.get_topic_terms(0,topn=10)

[(445, 0.0053695864),
 (372, 0.005282346),
 (151, 0.0049059745),
 (401, 0.0048675383),
 (140, 0.0046709445),
 (154, 0.0044971644),
 (2107, 0.0039803195),
 (80, 0.003914873),
 (1889, 0.00365126),
 (2150, 0.0031544494)]

In [55]:
dictionary[445]

'市场'

In [56]:
lda.get_term_topics('市场',minimum_probability=1e-4)

[(0, 0.0052932063),
 (1, 0.0062553575),
 (2, 0.0058568944),
 (3, 0.0015787539),
 (4, 0.005939167)]

不设置minimum_probability返回[]?  
解决参考：https://groups.google.com/forum/#!topic/gensim/y0hStr3uCGM

In [57]:
for topic in lda.print_topics(num_words=10):
    print(topic)

(0, '0.005*"市场" + 0.005*"都" + 0.005*"人" + 0.005*"中国" + 0.005*"不" + 0.004*"会" + 0.004*"永辉" + 0.004*"还" + 0.004*"提升" + 0.003*"超市"')
(1, '0.006*"市场" + 0.004*"不" + 0.004*"都" + 0.004*"乐视" + 0.004*"中国" + 0.004*"还" + 0.004*"会" + 0.004*"内容" + 0.003*"手机" + 0.003*"韩国"')
(2, '0.006*"市场" + 0.005*"不" + 0.005*"都" + 0.004*"企业" + 0.004*"会" + 0.004*"乐视" + 0.004*"内容" + 0.004*"还" + 0.003*"中国" + 0.003*"手机"')
(3, '0.006*"都" + 0.005*"人" + 0.005*"企业" + 0.005*"还" + 0.004*"中国" + 0.004*"冰架" + 0.004*"会" + 0.004*"跨境" + 0.004*"不" + 0.003*"配送"')
(4, '0.006*"会" + 0.006*"市场" + 0.005*"人" + 0.005*"记者" + 0.005*"都" + 0.004*"不" + 0.003*"还" + 0.003*"说" + 0.003*"内容" + 0.003*"中国"')


In [58]:
text = data["content"][102]

In [59]:
words = [[w.word for w in jp.cut(text) if w.flag in flags and w.word not in stop_words]]

In [60]:
text_corpus = [dictionary.doc2bow(word) for word in words]

In [61]:
list(lda[text_corpus])

[[(0, 0.0582345), (2, 0.17261589), (4, 0.76492655)]]

In [62]:
text

'新疆日报讯（通讯员吕丽娜报道）6月13日，笔者在拜城县残联培训基地见到了正在参加雕刻技术培训的托乎提·斯坎达尔，今年31岁的他是拜城县托克逊乡亚吐尔村的村民。通过学习，他已经可以利用残疾的双手雕刻出漂亮的葫芦花纹，学到技术后的他还加入了农民合作社，现在每个月有2000多元的稳定收入，最高时月收入达5000多元。\r\n据悉，今年以来，拜城县为进一步做好残疾人就业援助工作，采取“兴趣+技能”“培训+技能”等方式，在该县残疾人培训基地分别举办厨师班、刺绣班、雕刻班、地毯编织班等技能培训班，给全县的残疾人提供学习技能的机会，学习期间为他们免费提供食宿。\r\n现在每个班都会吸引一些残疾人前来学习技术，技能培训班不仅能帮助残疾人掌握一门技能，还能拓宽残疾人就业渠道。家住拜城县大桥乡农民阿不都热依木·亚森说：“我现在在合作社上班，有了稳定收入，家里的变化也非常大。谢谢党和政府的好政策！”\r\n截至目前，全县已有50多名残疾人成功实现就业。\r\n免责声明：本文仅代表作者个人观点，与环球网无关。其原创性以及文中陈述文字和内容未经本站证实，对本文以及其中全部或者部分内容、文字的真实性、完整性、及时性本站不作任何保证或承诺，请读者仅作参考，并请自行核实相关内容。\r\n'